# Family Facial Recognition for Home Security 
By Constantinos Skevofilax and Nikhil Sharma 

The goal of this project is to create a ML model that is able to classify members of a household over 'others', with the goal of integrating this model with a home security camera to determine who is entering a household and notifying the homeowner of who is visiting. 

In [8]:
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [11]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [13]:
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [14]:
# http://vis-www.cs.umass.edu/lfw/
# LFW Dataset for the 'others' 
!tar -xf lfw.tgz


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


## Data Augmentation 
We want to create a comparable family data set that will match the size of the negatives dataset. We will achieve this by using image augmentation methods such as flipping, stretching, and rotating images. comproable 

In [18]:
# Data Augmentation to match negative amount of 1555 
import urllib.request
import shutil
from IPython.display import Image

In [20]:
pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 22.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 707.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 5.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1
    Uninstalling torch-2.2.1:
      Successfully uninstalled torch-2.2.1

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the k

In [25]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from PIL import Image

In [209]:
from torchvision.transforms.functional import to_pil_image

def save_augmented_images(img_file, transform, save_path):
    # Load the requested image
    img = Image.open(img_file).resize((256, 256))

    # Apply transformations to the image
    augmented_images = [transform(img) for _ in range(4)]
    for i, augmented_img_tensor in enumerate(augmented_images):
        # Convert tensor to PIL Image
        augmented_img_pil = to_pil_image(augmented_img_tensor)
        # Save augmented image
        augmented_img_pil.save(os.path.join(save_path, f"{os.path.splitext(os.path.basename(img_file))[0]}_aug_{i}.jpg"))


In [210]:
def visualize_generator(img_file, transform):
    # Load the requested image
    img = Image.open(img_file).resize((256,256))

    # Apply transformations to the image
    imgs = [transform(img) for _ in range(4)]
    imgs = torch.stack(imgs)

    # Display the augmented images in a grid format
    grid = make_grid(imgs, nrow=2)

    # Convert to numpy for plotting
    grid_np = grid.numpy().transpose(1, 2, 0)
    grid_np = (grid_np * 255).astype(np.uint8)

    plt.figure(figsize=(15, 15))
    plt.axis("off")
    plt.imshow(grid_np)
    plt.show()

In [211]:
# Flips, "moves", adjusts brightness, and rotates family dataset to have a comparable number to the negatives 
# dataset size (1555 images)
transforms_list = [
    transforms.Compose([
        transforms.RandomAffine(degrees=0, translate=(0.5, 0)),
        transforms.Pad(padding=200, fill=0, padding_mode='edge'),
        transforms.ToTensor()
    ]),
    transforms.Compose([
        transforms.ColorJitter(brightness=(0, 1)),
        transforms.ToTensor()
    ]),
    transforms.Compose([
        transforms.RandomAffine(degrees=30),
        transforms.ToTensor()
    ])
]

In [214]:
POS_PATH

'data/positive'

In [217]:
# Get a list of subdirectories within the positive directory
subdirectories = [os.path.join(POS_PATH, sub_dir) for sub_dir in os.listdir(POS_PATH) if os.path.isdir(os.path.join(POS_PATH, sub_dir))]

# Loop through each subdirectory
for subdirectory in subdirectories:
    # Loop through each image file in the subdirectory
    for img_file in os.listdir(subdirectory):
        # Check if the file is an image file
        if img_file.endswith('.jpg'):
            # Get the full path to the image file
            img_path = os.path.join(subdirectory, img_file)
            # Apply each transformation set and visualize the result
            for transform in transforms_list:
                save_augmented_images(img_path, transform, subdirectory)

## Convolutional Neural Network

In [252]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
base_dir = '/root/Final_Project/data/'

# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # rescale pixel values and split data

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 1664 images belonging to 5 classes.
Found 414 images belonging to 5 classes.


In [253]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(250, 250, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # Assuming 4 family members + others
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [254]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # adjust based on your dataset size
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50  # adjust based on your dataset size
)

Epoch 1/15
 52/100 ━━━━━━━━━━━━━━━━━━━━ 42s 883ms/step - accuracy: 0.4382 - loss: 3.1084

2024-05-02 22:41:39.317349: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-05-02 22:41:41.661135: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: OSError: image file is truncated (16 bytes not processed)
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/site-packages/keras/s

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
OSError: image file is truncated (16 bytes not processed)
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 250, in _get_iterator
    for i, batch in enumerate(gen_fn()):

  File "/usr/local/lib/p

In [255]:
from PIL import Image
import os

# Define the directory containing your image files
directory = '/root/Final_Project/data/'

# Iterate over image files in the directory
for subdir in sorted(os.listdir(directory)):
    subdir_path = os.path.join(directory, subdir)
    if os.path.isdir(subdir_path):
        for img_file in os.listdir(subdir_path):
            img_path = os.path.join(subdir_path, img_file)
            try:
                # Attempt to open the image file
                with Image.open(img_path) as img:
                    # Image opened successfully, do further processing if needed
                    pass
            except OSError as e:
                # Handle the error (e.g., print error message or skip the image)
                print(f"Error processing image '{img_path}': {e}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define paths
base_dir = '/root/Final_Project/data/'

def generator_with_logging(generator):
    for i, (data, labels) in enumerate(generator):
        batch_filenames = [generator.filenames[idx] for idx in generator.index_array]
        print("Processing batch:", batch_filenames)
        yield data, labels


train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # rescale pixel values and split data

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical',
    subset='training')

train_generator = generator_with_logging(train_generator)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

validation_generator = generator_with_logging(validation_generator)

# Train the model with the modified generators
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # adjust based on your dataset size
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50  # adjust based on your dataset size
)


Found 1664 images belonging to 5 classes.
Found 414 images belonging to 5 classes.
Processing batch: ['george/george_0011_aug_0_aug_2.jpg', 'strangers/Gordon_Brown_0003.jpg', 'costaki/costaki_0013_aug_2_aug_3.jpg', 'costaki/costaki_0012.jpg', 'athena/athena_0007_aug_1_aug_1.jpg', 'strangers/Igor_Ivanov_0001.jpg', 'strangers/Edmund_Stoiber_0006.jpg', 'strangers/Donald_Rumsfeld_0022.jpg', 'strangers/Kofi_Annan_0013.jpg', 'athena/athena_0005_aug_2.jpg', 'strangers/Lance_Armstrong_0003.jpg', 'strangers/Gloria_Macapagal_Arroyo_0044.jpg', 'teresa/teresa_0013_aug_1_aug_2.jpg', 'athena/athena_0004_aug_2_aug_0.jpg', 'george/george_0015_aug_3_aug_3.jpg', 'strangers/Gerhard_Schroeder_0048.jpg', 'costaki/costaki_0010_aug_0_aug_0.jpg', 'george/george_0012_aug_2_aug_1.jpg', 'athena/athena_0012_aug_2.jpg', 'george/george_0010_aug_3.jpg', 'george/george_0006_aug_3_aug_1.jpg', 'strangers/Donald_Rumsfeld_0015.jpg', 'strangers/Gloria_Macapagal_Arroyo_0010.jpg', 'athena/athena_0014_aug_1.jpg', 'george/geo